<h1 style="font-size: 40px; text-align: center;">Causal Machine Learning Simulations</h1>

<h1 style="font-size: 20px; text-align: center;">(Original material by Michael C. Knaus)</h1>

<p align="center">
  <img src="https://pbs.twimg.com/media/FrGy6a5aMAAQ7u3?format=png&name=large" alt="Alt text" width="700" height="350">
</p>


# 4. Estimating constant effects: Double Selection to Double ML

$$
\text{Identifying Assumption 1 (Measured Confounding)}
$$

$$
Y(w) \perp\!\!\perp W \mid X \quad \text{for all} \quad W \in \mathbb{W} \subset \mathbb{R}
$$

$$
\text{Modelling Assumption 1 (Linear Potential Outcomes)}
$$

$$
Y(w) = \tau W + X'\beta + U_{Y(w)}; \quad E[U_{Y(w)} \mid X] = 0; \quad \forall w \in \mathbb{W}
$$

In [ ]:
# Import Libraries

import pandas as pd
import numpy as np
from IPython.display import Image

import graphviz as gr

import statsmodels.api as sm

from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_predict


In [ ]:
# Warnings

import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Restore warnings
warnings.filterwarnings("default")

import warnings

# Suppress FutureWarnings related to is_sparse
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.utils.validation")

# Reset the warnings filter if necessary
# warnings.resetwarnings()



$$
Y(w) = 2W + X_1 + 0.5 X_2 + U_{Y(w)} \quad \text{(True model)}
$$


$$
X_1 \sim \mathbb{U}(0,1)
$$
$$
X_2 \sim \mathbb{U}(0,20)
$$
$$
W \sim 0.9 \mathbb{U}(0,5) + 0.1 X_2
$$
$$
U_{Y(w)} \sim \mathbb{N}(0,I_n)
$$

In [ ]:
# DAGs to understand the problem

g = gr.Digraph()

g.edge("X1", "Y")
g.edge("X2", "Y")
g.edge("W", "Y")
g.edge("X2", "W")
g.node("X2","X2", color="red")

g

In [ ]:
# Data generating process

np.random.seed(21)

def create_data(N=200):
    df = pd.DataFrame({'X1': np.random.uniform(0, 1, size = N)})
    df['X2'] = np.random.uniform(0, 20, size = N)
    df['W'] = 0.9*np.random.uniform(0, 5, size=N) + 0.1*df['X2']
    df['epsilon'] = np.random.normal(size = N)
    df['Y'] = 2*df['W'] + df['X1'] + 0.5*df['X2'] + df['epsilon']
    return df

df = create_data(1000000)

## 4.1 OLS and omitted variable bias

$$
\hat{\beta}^{OLS} = \underset{\beta}{\text{arg min}} \sum_{i=1}^{N} \left(Y_i-\beta_0-\sum_{j=1}^p \beta_j X_{ij} \right)^2
$$

In [ ]:
# OLS 

Y = df['Y']
X = sm.add_constant(df[['W','X1','X2']])


model = sm.OLS(Y,X).fit()

print(model.summary())

In [ ]:
# OLS (omitted variable bias)

X = sm.add_constant(df[['W','X1']])

model2 = sm.OLS(Y,X).fit()

print(model2.summary())

## Note that in this model, to get an unbiased estimate of X1 we need an higher sample size (why?)

## 4.2 Double Selection

$$
Y(w) = 2W + X_1 + 0.5 X_2 - 0.05 X^2_2+ U_{Y(w)} \quad \text{(True model)}
$$

$$
X_1 \sim \mathbb{U}(0,1)
$$
$$
X_2 \sim \mathbb{U}(0,20)
$$
$$
W \sim 0.8\mathbb{U}(0,5) + 0.15 X_2 + 0.05 X^2_{2}
$$
$$
U_{Y(w)} \sim \mathbb{N}(0,I_n)
$$

In [ ]:
# Data generating process

np.random.seed(21)

def create_data(N=200):
    df = pd.DataFrame({'X1': np.random.uniform(0, 1, size = N)})
    df['X2'] = np.random.uniform(0, 20, size = N)
    df['W'] = 0.8*np.random.uniform(0, 5, size=N) + 0.15*df['X2'] + 0.05*df['X2']**2
    df['epsilon'] = np.random.normal(size = N)
    df['Y'] = 2*df['W'] + df['X1'] + 0.5*df['X2'] - 0.05*df['X2']**2 + df['epsilon']
    return df

df = create_data(100000)

In [ ]:
# OLS (now we get a biased estimate even if we include all the predictors)

Y = df['Y']
X = sm.add_constant(df[['W','X1','X2']])


model = sm.OLS(Y,X).fit()

print(model.summary())

1. Assume approximate sparsity ==> The number of controls with non-zero coefficients is small relative to the sample size
1. Select variables that predict outcome via Post-Lasso (w/o treatment variable)
2. Select variables that predict treatment via Post-Lasso
3. Use union of selected variables w/ treatment variable in OLS

$$
\text{Before applying LASSO, remember to scale the features!!}
$$

$$
\hat{\beta}^{Lasso} = \underset{\beta}{\text{arg min}} \sum_{i=1}^{N} \left(Y_i- \beta_0-\sum_{j=1}^p \beta_j X_{ij} \right)^2 + \lambda \sum_{j=1}^p \lvert \beta_j \rvert
$$

$$
\hat{\beta}^{\text{Post Lasso}} = \underset{\beta}{\text{arg min}}\sum_{i=1}^N \left(Y_i - \beta_0 - \sum_{j=1}^s \beta_j X^{sel}_{ij} \right)
$$

In [ ]:
poly = PolynomialFeatures(degree=2)

X = df[['X1','X2']]
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

df_array = poly.fit_transform(X_std)

feature_names = poly.get_feature_names_out(X.columns)

df_poly = pd.DataFrame(df_array, columns= feature_names)

df_poly.head()

## Note, it is important to scale features before applying LASSO because the penalty term is related to the module of the coefficients

In [ ]:
Y = df['Y']

X = df_poly[feature_names]

model_outcome = LassoCV().fit(X,Y)

alpha = model_outcome.alpha_

coefficients = model_outcome.coef_

In [ ]:
model_outcome.score(X,Y)

In [ ]:
print("alpha:", alpha)
print("coefficients:", coefficients)

In [ ]:
W = df['W']

model_treatment = LassoCV().fit(X,W)

alpha = model_treatment.alpha_

coefficients = model_treatment.coef_

In [ ]:
print("alpha:", alpha)
print("coefficients:", coefficients)

In [ ]:
Y = df['Y']

X = pd.DataFrame({
    'W': df['W'],
    'X2^2': df_poly['X2^2'],
    'X2': df['X2']
})

X = sm.add_constant(X)

model_ds = sm.OLS(Y,X).fit()

print(model_ds.summary())


## 4.3 Double ML: Partially Linear Model

$$
Y(w) = 2W + X_1 + 0.5 e^{X_2} + U_{Y(w)} \quad \text{(True model)}
$$

$$
X_1 \sim \mathbb{U}(0,1)
$$
$$
X_2 \sim \mathbb{U}(0,5)
$$
$$
W \sim \mathbb{U}(0,5) + 0.1e^{X_2}
$$
$$
U_{Y(w)} \sim \mathbb{N}(0,I_n)
$$

In [ ]:
np.random.seed(21)

def create_data(N=200):
    df = pd.DataFrame({'X1': np.random.uniform(0, 1, size = N)})
    df['X2'] = np.random.uniform(0, 5, size = N)
    df['W'] = np.random.uniform(0, 5, size=N) + 0.1*np.exp(df['X2'])  
    df['epsilon'] = np.random.normal(size = N)
    df['Y'] = 2*df['W'] + df['X1']  + 0.5* np.exp(df['X2']) + df['epsilon']  
    return df

df = create_data(10000)

In [ ]:
Y = df['Y']
X = sm.add_constant(df[['W','X1','X2']])


model = sm.OLS(Y,X).fit()

print(model.summary())

$$
\text{Modelling Assumption 2 (Partially Linear Potential Outcomes)}
$$

$$
Y(w) = \tau W + g(X) + U_{Y(w)}; \quad E[U_{Y(w)} \mid X] = 0; \quad \forall w \in \mathbb{W}
$$

$$
  \text{1. Form prediction model for the treatment:} \quad \hat{e}(X) \\ 
  \text{2. Form prediction model for the outcome:} \quad \hat{m}(X) \\
  \text{3. Run feasible residual-on-residual regression:} \quad \hat{\tau} = \underset{\tau}{\text{arg min}} \frac{1}{N} \sum_{i=1}^N \big(Y_i - \hat{m}(X_i)- \tau(W_i-\hat{e}(X_i))\big)^2 
$$

In [ ]:
from IPython.display import display, Image

# Define file paths for the images
file_paths = [
    r"C:\Users\feder\Desktop\Progetti Extra ML\Causal Machine Learning\4.  Folder Estimating constant effects Double Selection to Double ML\DML partially linear 1.png",
    r"C:\Users\feder\Desktop\Progetti Extra ML\Causal Machine Learning\4.  Folder Estimating constant effects Double Selection to Double ML\DML partially linear 2.png"
]

# Define width and height for each image
width = 1000
height = 600

# Create a list to store Image objects
images = []

# Load and display each image
for file_path in file_paths:
    image = Image(filename=file_path, width=width, height=height)
    images.append(image)

# Display images side by side
display(*images)


In [ ]:
# Predict E[W|X]
random_forest = RandomForestRegressor()

W = df['W']
X = df[['X1','X2']]

e_hat = cross_val_predict(random_forest, X, W)

In [ ]:
# Predict E[Y|X]
Y = df['Y']
X = df[['X1','X2']]

m_hat = cross_val_predict(random_forest, X, Y)


In [ ]:
y_res = Y - m_hat
w_res = W - e_hat

model_double_ML_partially_linear = sm.OLS(y_res,w_res).fit()

print(model_double_ML_partially_linear.summary())

## Notice that we don't have the constant